In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [46]:
class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        self.num_layers = len(layers)
        self.weights = [np.random.randn(y, x) for x, y in zip(layers[:-1], layers[1:])]
        self.biases = [np.random.randn(y, 1) for y in layers[1:]]

    def forward_propagation(self, a):
        for w, b in zip(self.weights, self.biases):
            a = self.activation(np.dot(w, a) + b)
        return a

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            self.update_weights_biases(X, y, learning_rate)

    def update_weights_biases(self, X, y, learning_rate):
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, target in zip(X, y):
            delta_nabla_w, delta_nabla_b = self.backpropagation(x, target)
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]

        self.weights = [w - (learning_rate / len(X)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (learning_rate / len(X)) * nb for b, nb in zip(self.biases, nabla_b)]

    def backpropagation(self, x, target):
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        activation = x
        activations = [x]
        zs = []

        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = self.activation(z)
            activations.append(activation)

        delta = self.cost_derivative(activations[-1], target) * self.activation_derivative(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = self.activation_derivative(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())

        return nabla_w, nabla_b

    def activation(self, z):
        return 1.0 / (1.0 + np.exp(-z))

    def activation_derivative(self, z):
        return self.activation(z) * (1 - self.activation(z))

    def cost_derivative(self, output_activation, target):
        return output_activation - target

In [48]:
# Create a dataset for XOR gate
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# Create a neural network with 2 input neurons, 2 hidden neurons, and 1 output neuron
network = NeuralNetwork([2, 2, 1])

# Train the network
network.train(X, y, epochs=10000, learning_rate=0.1)

# Test the network
for i in range(len(X)):
    input_data = X[i]
    target = y[i]
    output = network.forward_propagation(input_data)
    print("Input:", input_data, "Target:", target, "Output:", output)


Input: [0 0] Target: [0] Output: [[0.08692921 0.16123309]]
Input: [0 1] Target: [1] Output: [[0.88862752 0.79005974]]
Input: [1 0] Target: [1] Output: [[0.86074064 0.81294961]]
Input: [1 1] Target: [0] Output: [[0.15311607 0.17361114]]
